# LIXO 
---

OCUPANTES

In [ ]:
async def run(self):
            '''
            se saida-> vai para saida
            se não random
            '''

            if self.clossest_exit()!= -1:
                EvacuationUI.update_agent_position(self.agent.name,self.clossest_exit())

            if self.agent.exits["Exit 1"] == 'closed':
                print(f"{self.agent.name} found Exit 1 clsoed, rerouting to another exit...")

            else:
                print(f"{self.agent.name} is heading to Exit 1")

            if self.agent.elevator == 'off':
                print(f"{self.agent.name} found the elevator off, will try to use stairs")


            await asyncio.sleep(2)

---

BUILDING_AGENT

----

EMERGENCY_RESPONDER

---